In [9]:
import tensorflow as tf 
import cv2 
import os 

In [10]:
rootDir = 'A:\Lip Reading\Potential Datasets\BBC LRS2\\allFiles'
vocab = [x for x in "ABCDEFGHIJKLMNOPQRSTUVWXYZ'?!123456789 "]

def getFrameCount(path) -> int: 
    cap = cv2.VideoCapture(path)
    frameCount = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    cap.release()
    return frameCount
def getCharCount(path) -> int: 
    return len(loadText(path))
def loadText(path): 
    # open and parse the file 
    with open(path, 'r') as file: lines = file.readlines()
    file.close()
    
    # return the number equivalent of each of the characters of the word 
    tokens = []
    words = lines[0].split()
    del words[0]
    for word in words: tokens = [*tokens,' ', word]
    
    return charToNum(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))[1:]   
charToNum = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")

In [34]:
# finding the max number of frames & chars in a certain video 

def dataStats(dir): 
    frameCt = []
    charCt = []
    vidList = []
    textList = []
    # iterating through each file in the root folder
    for subdir, dirs, files in os.walk(dir):
        for file in files:
            if file.split('.')[1] == "mp4": 
                frameCt.append(getFrameCount(os.path.join(subdir, file)))
                vidList.append(os.path.join(subdir, file))
            if file.split('.')[1] == "txt": 
                charCt.append(getCharCount(os.path.join(subdir, file)))
                textList.append(os.path.join(subdir, file))
    return frameCt, charCt, vidList, textList
        

In [35]:
def mostCommon(lst):
    return max(set(lst), key=lst.count)
def printStats(frameCt, charCt, vidList, textList):    
    print("max frame count:", max(frameCt), ", index:", frameCt.index(max(frameCt)))
    print("longest video:", vidList[frameCt.index(max(frameCt))])
    print("max char count:", max(charCt), ", index:", charCt.index(max(charCt)))
    print("longest text:", textList[charCt.index(max(charCt))])
    
    print("=" * 100)
    
    print("min frame count:", min(frameCt), ", index:", frameCt.index(min(frameCt)))
    print("shortest video:", vidList[frameCt.index(min(frameCt))])
    print("min char count:", min(charCt), ", index:", charCt.index(min(charCt)))
    print("shortest text:", textList[charCt.index(min(charCt))])
    
    print("=" * 100)
    
    print("total num of frames:", sum(charCt))
    print("avg num of frames: ", sum(frameCt)/len(frameCt))
    print("avg chars per vid:", sum(charCt)/len(charCt))
    print("most common number of  frames:", mostCommon(frameCt) )


In [38]:
frameCt, charCt, vidList, textList = dataStats("A:\Lip Reading\Potential Datasets\BBC LRS2\\trainFiles")

printStats(frameCt, charCt, vidList, textList)

max frame count: 115.0 , index: 4
longest video: A:\Lip Reading\Potential Datasets\BBC LRS2\trainFiles\5536745420943636139_00027.mp4
max char count: 80 , index: 19
longest text: A:\Lip Reading\Potential Datasets\BBC LRS2\trainFiles\6383716407688597209_00035.txt
min frame count: 21.0 , index: 10
shortest video: A:\Lip Reading\Potential Datasets\BBC LRS2\trainFiles\6382974237339848372_00005.mp4
min char count: 15 , index: 7
shortest text: A:\Lip Reading\Potential Datasets\BBC LRS2\trainFiles\6382931717163617970_00008.txt
total num of frames: 699
avg num of frames:  47.916666666666664
avg chars per vid: 29.125
most common number of  frames: 24.0
